# 드롭아웃
- 데이터 증강 기법을 통해서 데이터에 노이즈를 추가함으로써 모델이 불필요한 특징들을 학습하는 것을 방지했다. 이것은 신경망의 입력 또는 출력 위치에 노이즈를 추가한 것이다.
- 그렇다면 이제 신경망의 중간에 노이즈를 추가하는 방법을 알아보도록 하겠다.

## 드롭아웃의 동작 방식
- 드롭아웃은 임의의 노드를 일정 확률로 drop해서 학습에 참여하지 않도록 하는 방법이다
- 데이터를 신경망에 통과시키는 과정에서 노드가 생략되므로 자연스럽게 역전파에서도 제외된다.
- ![](../img/dropout.png)
- 드롭되는 노드는 매 미니배치마다 이항분포를 활용하여 랜덤으로 선정된다. 
- 이때 노드의 드롭 확률 p가 하이퍼파라미터가 되어 사용자가 최적의 설정값을 찾아주어야 한다.

## 학습과 추론 방식의 차이
- 드롭아웃의 동작 방식은 학습에서만 적용되는 방법이다.
- 추론에서는 드롭되는 노드없이 모든 노드가 항상 추론에 참여한다.
- 이때 중요한 점은 가중치 파라미터 W에 (1-p)를 곱해주어야 한다는 것이다.
- 추론할 때에는 다음 계층은 항상 전체 노드로부터 값을 전달받을 것이다.
- 그러므로 추론할 때에는 가중치 파라미터에 (1-p)을 곱해주어 이것을 상쇄해야한다.
- 파이토치에서는 조금 다르게 구현되어 있다. 학습할 때에 1/1-p를 드롭아웃 계층 출력값에 곱해주어 학습과 추론 사이의 차이를 없앨 수 있다.

## 드롭아웃의 구현
- 드롭아웃은 신경망의 계층으로 구현할 수 있다.
- 파이토치에서는 미리 구현된 으롭아웃 계층을 제공하고 있다.
- 드롭아웃이 구현되는 위치는 활성 함수와 다음 계층 사이가 된다.
- 보통 신경망의 양 끝단인 입력 계층 이전과 출력 계층 이후에는 드롭아웃이 적용되지 않는다.

In [3]:
import torch.nn as nn
p = 0.3

net = nn.Sequential(
    nn.Linear(300,200),
    nn.LeakyReLU(),
    nn.Dropout(p),
    nn.Linear(200,100),
    nn.LeakyReLU(),
    nn.Dropout(p),
    nn.Linear(100,50),
    nn.LeakyReLU(),
    nn.Dropout(p),
    nn.Linear(50, 10)

)
net

Sequential(
  (0): Linear(in_features=300, out_features=200, bias=True)
  (1): LeakyReLU(negative_slope=0.01)
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=200, out_features=100, bias=True)
  (4): LeakyReLU(negative_slope=0.01)
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=100, out_features=50, bias=True)
  (7): LeakyReLU(negative_slope=0.01)
  (8): Dropout(p=0.3, inplace=False)
  (9): Linear(in_features=50, out_features=10, bias=True)
)

- 또한 추론과 학습이 다르게 동작해야 하기 때문에 추론 코드에 추가적인 구현이 필요하다.
- 파이토치 nn.Module을 상속받은 클래스는 train()과 eval()이라는 메서드를 제공하여 모델의 학습 모드와 추론 모드를 쉽게 왔다갔다 할 수 있도록한다.
- train()과 eval()이 호출되면 해당 객체 안에 포함된 드롭아웃과 같이 학습과 추론에서 다르게 동작하는 계층들은 자동으로 학습모드와 추론 모드로 전환된다. 
- 물론 드랍아웃 자체도 nn.Module을 상속받은 클래스의 객체일테니 train()과 eval()을 통해 직접 모드를 전환할 수 있다.

In [ ]:
for epoch_idx in range(n_epochs):
    net.train()
    for iter_idx in range(n_train_iters):
        # Training procedure
    
    net.eval()
    for iter_idx in range(n_valid_iters):
        # Validation procedure


> 항상 기본적으로 모드 변환 코드를 집어넣을 것을 권장한다.

## 드롭아웃의 의미와 한계
- 학습을 방해하는 방식으로 동작하기 때문에 드롭아웃이 적용되면 비록 일반화 성능이 개선될 수는 있어도 손실 값의 수렵 속도가 저하도리 수 있고 학습 오차가 증가할 수 있다.
- 튜닝은 보통 0.1에서 0.5사이의 0.1단위 값으로 튜닝한다.